# Semesterprojekt: Retourenvorhersage

|Paul|Antonio|Tim|Flo|
|---|---|---|---|
|hopa1029|coan1014|ehti1011|klfl1012|
80001|       |80055|79679|

**Gliederung:**
1. Data Cleaning
   
2. Klassifikaitonsmodelle mit Hyperparameteroptimierung:
   - 2.1 Modell a 
  
   - 2.2 Modell b
  
   - 2.3 Modell c 
                  
3. Evaluation mit ROC Plot 

4. Aufgabe 5

## 1. Data Cleaning

## 2. Klassifikationsmodelle mit Hyperparameteroptimierung

### 2.1 Modell a

### 2.2 Modell b

### 2.3 Modell c

## 3. Evaluation mit ROC Plot

## 4. Aufgabe 5